In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
# 모델
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

In [4]:
gens = pd.read_csv("../gens.csv")
p_gens = pd.read_csv("../pred.csv")
wf = pd.read_csv("../weather_forecast.csv")

In [6]:
real_dict = {"d25" : [0, 0, 0, 0, 0, 0,
                      0, 10, 31, 54, 70, 72, 78, 73, 63, 40, 20, 3, 0,
                      0, 0, 0, 0, 0],
             "d26" : [0, 0, 0, 0, 0, 0,
                      1, 3, 11, 40, 61, 52, 75, 61, 62, 40, 18, 2, 0,
                      0, 0, 0, 0, 0],
             "d27" : [0, 0, 0, 0, 0, 0,
                      1, 9, 36, 46, 53, 74, 89, 81, 64, 54, 23, 4, 0,
                      0, 0, 0, 0, 0],
             "d28" : [0, 0, 0, 0, 0, 0,
                      0, 5, 29, 56, 67, 82, 78, 72, 63, 46, 17, 3, 0,
                      0, 0, 0, 0, 0],
             "d29" : [0, 0, 0, 0, 0, 0,
                      0, 4, 20, 56, 66, 76, 53, 57, 66, 42, 20, 2, 0,
                      0, 0, 0, 0, 0],
             "d30" : [0, 0, 0, 0, 0, 0,
                      0, 4, 20, 56, 65, 79, 81, 75, 62, 45, 22, 3, 0,
                      0, 0, 0, 0, 0],
             "d31" : [0, 0, 0, 0, 0, 0,
                      0, 4, 8, 22, 37, 76, 73, 75, 58, 44, 22, 4, 0,
                      0, 0, 0, 0, 0]}

In [3]:
## test data set 만드는 함수
def make_test_xy(wf,gf):
    gf.columns = ["time","m0","m1","m2","m3","m4"]
    train = pd.merge(gf,wf,on="time")
    train["time"] = pd.to_datetime(train["time"])
    train["month"]=train["time"].dt.month
    train["day"]=train["time"].dt.day
    train["hour"]=train["time"].dt.hour
    
    train_1 = train[(train["hour"]<=10) & (train["hour"]>=7)]
    train_2 = train[(train["hour"]<=14) & (train["hour"]>=11)]
    train_3 = train[(train["hour"]<=19) & (train["hour"]>=15)]
    
    train11 = train_1[["m0","m1","m2","m3","uv_idx","elevation"]]
    train12 = train_2[["m0","m1","m2","m3","uv_idx","elevation"]]
    train13 = train_3[["m0","m1","m2","m3","uv_idx","elevation"]]
    train21 = train_1.drop(["m0","m1","m2","m3","m4","uv_idx","elevation","time","hour"],axis=1)
    train22 = train_2.drop(["m0","m1","m2","m3","m4","uv_idx","elevation","time","hour"],axis=1)
    train23 = train_3.drop(["m0","m1","m2","m3","m4","uv_idx","elevation","time","hour"],axis=1)
    
    return train11,train12,train13,train21,train22,train23

# 10-30일껄로 예측 진행

In [117]:
# 예측에 사용할 데이터 셋 불러오기 
x_pred_ori = pd.read_csv("../data/gen_fcst_10_2023-10-31.csv")
x_pred_wf = pd.read_csv("../data/weather_forecasts/wf10_2023-10-31.csv")

In [118]:
day = real_dict["d31"]

In [119]:
# 함수에 넣어서 데이터셋 분리
test11,test12,test13,test21,test22,test23 = make_test_xy(x_pred_wf,x_pred_ori)

In [86]:
import joblib

In [120]:
model11=joblib.load("../model/시간1회귀fitting_r1.pkl")
model12=joblib.load("../model/시간2회귀fitting_r1.pkl")
model13=joblib.load("../model/시간3회귀fitting_r1.pkl")
# 예측 진행 (1차 모델링 y_hat)
y_pred11=model11.predict(test11)
y_pred12=model12.predict(test12)
y_pred13=model13.predict(test13)
model21=joblib.load("../model/시간1잔차fitting_r1.pkl")
model22=joblib.load("../model/시간2잔차fitting_r1.pkl")
model23=joblib.load("../model/시간3잔차fitting_r1.pkl")
y_pred21 = model21.predict(test21)
y_pred22 = model22.predict(test22)
y_pred23 = model23.predict(test23)
result1 = y_pred11+y_pred21
result2 = y_pred12+y_pred22
result3 = y_pred13+y_pred23
result = np.concatenate((result1,result2,result3))
res1 = result

In [121]:
res1

array([ 0.7379158 ,  7.80662342, 24.67734285, 47.58976551, 68.77934771,
       77.50234766, 76.2886703 , 72.73994815, 58.47561335, 38.65736662,
       15.51367102,  2.15816336, -0.52915807])

In [122]:
# 예측에 사용할 데이터 셋 불러오기 
x_pred_ori = pd.read_csv("../data/gen_fcst_17_2023-10-31.csv")
x_pred_wf = pd.read_csv("../data/weather_forecasts/wf17_2023-10-31.csv")
# 함수에 넣어서 데이터셋 분리
test11,test12,test13,test21,test22,test23 = make_test_xy(x_pred_wf,x_pred_ori)

In [123]:
model11=joblib.load("../model/시간1회귀fitting_r2.pkl")
model12=joblib.load("../model/시간2회귀fitting_r2.pkl")
model13=joblib.load("../model/시간3회귀fitting_r2.pkl")
# 예측 진행 (1차 모델링 y_hat)
y_pred11=model11.predict(test11)
y_pred12=model12.predict(test12)
y_pred13=model13.predict(test13)
model21=joblib.load("../model/시간1잔차fitting_r2.pkl")
model22=joblib.load("../model/시간2잔차fitting_r2.pkl")
model23=joblib.load("../model/시간3잔차fitting_r2.pkl")
y_pred21 = model21.predict(test21)
y_pred22 = model22.predict(test22)
y_pred23 = model23.predict(test23)
result1 = y_pred11+y_pred21
result2 = y_pred12+y_pred22
result3 = y_pred13+y_pred23
result = np.concatenate((result1,result2,result3))
res2 = result

In [124]:
res2

array([ 1.00101147,  7.14948782, 25.76676122, 50.46144822, 65.49114253,
       75.9568201 , 73.9068651 , 71.49869172, 57.78679825, 40.65899505,
       16.79421644,  3.46914798,  0.65229753])

In [125]:
pred = (res1+res2)/2

In [126]:
pred

array([8.69463632e-01, 7.47805562e+00, 2.52220520e+01, 4.90256069e+01,
       6.71352451e+01, 7.67295839e+01, 7.50977677e+01, 7.21193199e+01,
       5.81312058e+01, 3.96581808e+01, 1.61539437e+01, 2.81365567e+00,
       6.15697268e-02])

In [127]:
pred = [0,0,0,0,0,0, 8.69463632e-01, 7.47805562e+00, 2.52220520e+01, 4.90256069e+01,
       6.71352451e+01, 7.67295839e+01, 7.50977677e+01, 7.21193199e+01,
       5.81312058e+01, 3.96581808e+01, 1.61539437e+01, 2.81365567e+00,
       6.15697268e-02,0,0,0,0,0]

In [114]:
import numpy as np

def testMSEnIncentive(pred,day):
    realGens1030 = np.array(day)    
    
    rss = np.power(pred - realGens1030, 2)
    rss1 = rss[6:10]
    rss2 = rss[10:14]
    rss3 = rss[14:19]
    mse1 = sum(rss1/len(rss1))
    mse2 = sum(rss2/len(rss2))
    mse3 = sum(rss3/len(rss3))
    mse = [mse1,mse2,mse3]

    score_df = abs(pred - realGens1030)*100/99
    score = []
    for idx, i in enumerate(score_df):
        if i <=6:
            score.append(realGens1030[idx] * 4)
        elif (i <=8) & (i > 6):
            score.append(realGens1030[idx] * 3)
        else:
            score.append(0)
    score1 = score[6:10]
    score2 = score[10:14]
    score3 = score[14:19]
    final_score = [score1,score2,score3]
    
    sum_score = sum(score)

    return mse, final_score , score, sum_score

In [128]:
testMSEnIncentive(pred,day)

([259.958835326804, 230.34105926647825, 10.891237378158674],
 [[0, 16, 0, 0], [0, 304, 292, 300], [232, 176, 88, 16, 0]],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  16,
  0,
  0,
  0,
  304,
  292,
  300,
  232,
  176,
  88,
  16,
  0,
  0,
  0,
  0,
  0,
  0],
 1424)